In [1]:
%pip install -I scikit-learn==0.24.1

     |████████████████████████████████| 22.3MB 6.3MB/s 
     |████████████████████████████████| 27.4MB 100kB/s 
     |████████████████████████████████| 15.3MB 281kB/s 
     |████████████████████████████████| 307kB 40.9MB/s 
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.20.3 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [1]:
%matplotlib notebook
import plotly.express as px
import numpy as np
# import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.base import TransformerMixin
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.kernel_approximation import Nystroem
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_curve, auc
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.preprocessing import label_binarize
from sklearn.cluster import DBSCAN, KMeans, SpectralClustering, AgglomerativeClustering


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
df_crime = pd.read_csv('gdrive/MyDrive/Colab Notebooks/crime.csv', index_col=0)
df_crime['Date'] = df_crime['Date'].apply(lambda x: datetime.datetime.strptime(x, "%m/%d/%Y %I:%M:%S %p" ))
df_crime

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:583: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards,Boundaries - ZIP Codes,Police Districts,Police Beats
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10224738,HY411648,2015-09-05 13:30:00,043XX S WOOD ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,924,9.0,12.0,61.0,08B,1165074.0,1875917.0,2015,02/10/2018 03:50:01 PM,41.815117,-87.670000,"(41.815117282, -87.669999562)",29.0,14924.0,59.0,706.0,3.0,37.0,23.0,108.0
10224739,HY411615,2015-09-04 11:30:00,008XX N CENTRAL AVE,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,1511,15.0,29.0,25.0,06,1138875.0,1904869.0,2015,02/10/2018 03:50:01 PM,41.895080,-87.765400,"(41.895080471, -87.765400451)",4.0,4299.0,26.0,562.0,45.0,5.0,25.0,67.0
11646166,JC213529,2018-09-01 00:01:00,082XX S INGLESIDE AVE,0810,THEFT,OVER $500,RESIDENCE,False,True,631,6.0,8.0,44.0,06,NaN,NaN,2018,04/06/2019 04:04:43 PM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10224740,HY411595,2015-09-05 12:45:00,035XX W BARRY AVE,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,1412,14.0,35.0,21.0,18,1152037.0,1920384.0,2015,02/10/2018 03:50:01 PM,41.937406,-87.716650,"(41.937405765, -87.716649687)",15.0,21538.0,22.0,216.0,12.0,39.0,7.0,168.0
10224741,HY411610,2015-09-05 13:00:00,0000X N LARAMIE AVE,0560,ASSAULT,SIMPLE,APARTMENT,False,True,1522,15.0,28.0,25.0,08A,1141706.0,1900086.0,2015,02/10/2018 03:50:01 PM,41.881903,-87.755121,"(41.881903443, -87.755121152)",11.0,22216.0,26.0,696.0,23.0,32.0,25.0,81.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12361044,JE221973,2021-05-05 09:02:00,056XX S PARKSIDE AVE,0810,THEFT,OVER $500,ALLEY,False,False,811,8.0,23.0,56.0,06,1139664.0,1866666.0,2021,05/12/2021 04:59:22 PM,41.790231,-87.763434,"(41.790231434, -87.763433653)",35.0,22268.0,53.0,590.0,6.0,7.0,13.0,131.0
12359057,JE221479,2021-05-05 18:29:00,016XX N MILWAUKEE AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,DRUG STORE,False,False,1434,14.0,2.0,24.0,11,1162627.0,1910739.0,2021,05/12/2021 04:59:22 PM,41.910724,-87.678001,"(41.910723622, -87.67800053)",16.0,22535.0,25.0,298.0,11.0,1.0,7.0,192.0
12358197,JE221162,2021-05-05 17:00:00,052XX W CONGRESS PKWY,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,False,False,1522,15.0,29.0,25.0,07,1141620.0,1897182.0,2021,05/12/2021 04:59:22 PM,41.873936,-87.755509,"(41.873936087, -87.755508739)",52.0,22216.0,26.0,68.0,7.0,32.0,25.0,97.0


In [4]:
df_inc = pd.read_csv('gdrive/MyDrive/Colab Notebooks/wealth.csv')
df_inc

,Community Area Number,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
0,1.0,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
1,2.0,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46.0
2,3.0,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20.0
3,4.0,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17.0
4,5.0,North Center,0.3,7.5,5.2,4.5,26.2,57123,6.0
...,...,...,...,...,...,...,...,...,...
73,74.0,Mount Greenwood,1.0,3.4,8.7,4.3,36.8,34381,16.0
74,75.0,Morgan Park,0.8,13.2,15.0,10.8,40.3,27149,30.0
75,76.0,O'Hare,3.6,15.4,7.1,10.9,30.3,25828,24.0
76,77.0,Edgewater,4.1,18.2,9.2,9.7,23.8,33385,19.0


In [5]:
df_crime_short = df_crime[(df_crime['Date'] >= np.datetime64('2008-01-01')) & (df_crime['Date'] <= np.datetime64('2012-01-01'))].copy()
df_crime_short

,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards,Boundaries - ZIP Codes,Police Districts,Police Beats
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10225648,HY412901,2011-08-01 08:00:00,080XX S TRUMBULL AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,834,8.0,18.0,70.0,11,1154779.0,1851149.0,2011,02/09/2018 03:44:29 PM,41.747362,-87.708424,"(41.747362057, -87.708423712)",6.0,4300.0,69.0,199.0,30.0,8.0,13.0,232.0
11042125,JA376558,2011-12-16 00:00:00,003XX W 64TH ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,HOSPITAL BUILDING/GROUNDS,False,False,722,7.0,20.0,68.0,11,NaN,NaN,2011,08/05/2017 03:50:08 PM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11042582,JA377037,2011-01-01 00:01:00,054XX S CALIFORNIA AVE,1754,OFFENSE INVOLVING CHILDREN,AGG SEX ASSLT OF CHILD FAM MBR,APARTMENT,True,True,923,9.0,14.0,63.0,02,NaN,NaN,2011,08/13/2017 03:50:54 PM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11042930,JA374409,2010-01-01 00:01:00,080XX S MARSHFIELD AVE,0266,CRIM SEXUAL ASSAULT,PREDATORY,RESIDENCE,False,False,611,6.0,21.0,71.0,02,NaN,NaN,2010,08/05/2017 03:50:08 PM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11042914,JA374555,2008-01-01 00:01:00,011XX S WHIPPLE ST,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,RESIDENCE,False,True,1134,11.0,28.0,29.0,02,NaN,NaN,2008,08/19/2017 03:48:07 PM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8093964,HT327936,2011-06-03 03:00:00,073XX S STEWART AVE,0281,CRIMINAL SEXUAL ASSAULT,NON-AGGRAVATED,APARTMENT,False,True,731,7.0,6.0,69.0,02,1174894.0,1856329.0,2011,05/06/2021 04:48:14 PM,41.761152,-87.634562,"(41.761152087, -87.634562414)",17.0,21559.0,66.0,511.0,32.0,11.0,17.0,214.0
12359222,JE222451,2010-11-13 12:00:00,022XX S WHIPPLE ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,OTHER (SPECIFY),False,False,1033,10.0,12.0,30.0,11,NaN,NaN,2010,05/07/2021 04:49:27 PM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7977063,HT207697,2009-12-01 00:00:00,021XX W 19TH ST,1754,OFFENSE INVOLVING CHILDREN,AGGRAVATED SEXUAL ASSAULT OF CHILD BY FAMILY M...,APARTMENT,False,True,1223,12.0,25.0,31.0,02,NaN,NaN,2009,05/12/2021 04:57:30 PM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# change train/test data size
data_coef = 0.3
data = df_crime_short[['Date', 'Community Areas', 'Primary Type']].sample(int(df_crime_short.shape[0]*data_coef))
data

,Date,Community Areas,Primary Type
ID,,,
6986604,2009-06-24 02:53:00,19.0,NARCOTICS
8301170,2011-10-09 21:20:00,26.0,BATTERY
6532424,2008-09-09 12:44:00,22.0,THEFT
8360064,2011-11-17 07:35:00,24.0,ASSAULT
7426569,2010-03-27 21:00:00,26.0,NARCOTICS
...,...,...,...
7720493,2010-09-18 12:00:00,45.0,THEFT
8351351,2011-11-11 12:15:00,32.0,BATTERY
7857450,2010-12-22 01:38:00,26.0,NARCOTICS


In [7]:
data['year'] = data['Date'].apply(lambda x: x.year)
data['month'] = data['Date'].apply(lambda x: x.month)
data['day'] = data['Date'].apply(lambda x: x.day)
data['hour'] = data['Date'].apply(lambda x: x.hour)
data

,Date,Community Areas,Primary Type,year,month,day,hour
ID,,,,,,,
6986604,2009-06-24 02:53:00,19.0,NARCOTICS,2009,6,24,2
8301170,2011-10-09 21:20:00,26.0,BATTERY,2011,10,9,21
6532424,2008-09-09 12:44:00,22.0,THEFT,2008,9,9,12
8360064,2011-11-17 07:35:00,24.0,ASSAULT,2011,11,17,7
7426569,2010-03-27 21:00:00,26.0,NARCOTICS,2010,3,27,21
...,...,...,...,...,...,...,...
7720493,2010-09-18 12:00:00,45.0,THEFT,2010,9,18,12
8351351,2011-11-11 12:15:00,32.0,BATTERY,2011,11,11,12
7857450,2010-12-22 01:38:00,26.0,NARCOTICS,2010,12,22,1


In [8]:
data = data.dropna()
data['Community Areas'] = data['Community Areas'].apply(lambda x: int(x))
data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Date,Community Areas,Primary Type,year,month,day,hour
ID,,,,,,,
6986604,2009-06-24 02:53:00,19,NARCOTICS,2009,6,24,2
8301170,2011-10-09 21:20:00,26,BATTERY,2011,10,9,21
6532424,2008-09-09 12:44:00,22,THEFT,2008,9,9,12
8360064,2011-11-17 07:35:00,24,ASSAULT,2011,11,17,7
7426569,2010-03-27 21:00:00,26,NARCOTICS,2010,3,27,21
...,...,...,...,...,...,...,...
7720493,2010-09-18 12:00:00,45,THEFT,2010,9,18,12
8351351,2011-11-11 12:15:00,32,BATTERY,2011,11,11,12
7857450,2010-12-22 01:38:00,26,NARCOTICS,2010,12,22,1


In [9]:
data = data.set_index('Community Areas').join(df_inc.set_index('Community Area Number'), how='inner').reset_index().rename({'index':'Community Area Number'}, axis=1)
data

,Community Area Number,Date,Primary Type,year,month,day,hour,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
0,1,2009-11-20 13:00:00,DECEPTIVE PRACTICE,2009,11,20,13,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
1,1,2008-08-25 16:07:48,NARCOTICS,2008,8,25,16,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
2,1,2009-07-17 16:30:00,BATTERY,2009,7,17,16,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
3,1,2011-05-22 17:20:00,BATTERY,2011,5,22,17,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
4,1,2011-09-10 01:00:00,BATTERY,2011,9,10,1,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456971,77,2010-06-30 18:30:00,DECEPTIVE PRACTICE,2010,6,30,18,Edgewater,4.1,18.2,9.2,9.7,23.8,33385,19.0
456972,77,2008-12-19 18:45:00,CRIMINAL DAMAGE,2008,12,19,18,Edgewater,4.1,18.2,9.2,9.7,23.8,33385,19.0
456973,77,2008-12-27 01:46:40,CRIMINAL DAMAGE,2008,12,27,1,Edgewater,4.1,18.2,9.2,9.7,23.8,33385,19.0
456974,77,2009-10-19 13:00:00,DECEPTIVE PRACTICE,2009,10,19,13,Edgewater,4.1,18.2,9.2,9.7,23.8,33385,19.0


In [10]:
stats_border = 4
stats = data['Primary Type'].value_counts()
stats.head(stats_border)

THEFT              94547
BATTERY            80244
CRIMINAL DAMAGE    53205
NARCOTICS          50671
Name: Primary Type, dtype: int64

In [11]:
data_sl = data.set_index('Primary Type', drop=True).join(
    pd.DataFrame(stats.head(stats_border)).drop('Primary Type', axis=1),
    how = 'inner').reset_index()
data_sl

,index,Community Area Number,Date,year,month,day,hour,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
0,BATTERY,1,2009-07-17 16:30:00,2009,7,17,16,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
1,BATTERY,1,2011-05-22 17:20:00,2011,5,22,17,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
2,BATTERY,1,2011-09-10 01:00:00,2011,9,10,1,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
3,BATTERY,1,2010-01-28 18:45:00,2010,1,28,18,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
4,BATTERY,1,2008-05-29 00:00:00,2008,5,29,0,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278662,THEFT,77,2010-02-01 07:20:00,2010,2,1,7,Edgewater,4.1,18.2,9.2,9.7,23.8,33385,19.0
278663,THEFT,77,2009-10-29 16:30:00,2009,10,29,16,Edgewater,4.1,18.2,9.2,9.7,23.8,33385,19.0
278664,THEFT,77,2011-10-30 01:00:00,2011,10,30,1,Edgewater,4.1,18.2,9.2,9.7,23.8,33385,19.0
278665,THEFT,77,2008-12-22 22:00:00,2008,12,22,22,Edgewater,4.1,18.2,9.2,9.7,23.8,33385,19.0


In [12]:
data_sl['index'] = LabelEncoder().fit_transform(data_sl['index'])
data_sl['index'].value_counts()

3    94547
0    80244
1    53205
2    50671
Name: index, dtype: int64

In [13]:
data_sl.drop(['index', 'Community Area Number', 'COMMUNITY AREA NAME', 'Date'],axis=1).columns

Index(['year', 'month', 'day', 'hour', 'PERCENT OF HOUSING CROWDED',
       'PERCENT HOUSEHOLDS BELOW POVERTY', 'PERCENT AGED 16+ UNEMPLOYED',
       'PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA',
       'PERCENT AGED UNDER 18 OR OVER 64', 'PER CAPITA INCOME ',
       'HARDSHIP INDEX'],
      dtype='object')

In [14]:
# downsampling
classes = data_sl['index'].unique()
min_y = 10000000
for cls in classes:
  min_y = min(min_y, data_sl[data_sl['index'] == cls].shape[0])
res = []
for cls in classes:
  res.append(data_sl[data_sl['index'] == cls].sample(min_y))
data_sl = pd.concat(res, ignore_index=True)
data_sl

,index,Community Area Number,Date,year,month,day,hour,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
0,0,57,2011-01-19 20:40:00,2011,1,19,20,Archer Heights,8.5,14.1,16.5,35.9,39.2,16134,67.0
1,0,20,2011-08-01 12:30:00,2011,8,1,12,Hermosa,6.9,20.5,13.1,41.6,36.4,15089,71.0
2,0,69,2011-04-25 01:30:00,2011,4,25,1,Greater Grand Crossing,3.6,29.6,23.0,16.5,41.0,17285,66.0
3,0,31,2010-08-18 20:52:00,2010,8,18,20,Lower West Side,9.6,25.8,15.8,40.7,32.6,16444,76.0
4,0,63,2010-06-14 09:20:14,2010,6,14,9,Gage Park,15.8,23.4,18.2,51.5,38.8,12171,93.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202679,3,16,2010-12-11 17:15:00,2010,12,11,17,Irving Park,6.3,13.1,10.0,22.4,31.6,27249,34.0
202680,3,7,2009-05-27 11:25:00,2009,5,27,11,Lincoln Park,0.8,12.3,5.1,3.6,21.5,71551,2.0
202681,3,68,2008-09-29 08:30:00,2008,9,29,8,Englewood,3.8,46.6,28.0,28.5,42.5,11888,94.0
202682,3,65,2010-03-01 09:00:00,2010,3,1,9,West Lawn,5.8,14.9,9.6,33.6,39.6,16907,56.0


In [15]:
data_x_train, data_x_test, data_y_train, data_y_test = train_test_split(
        # data.drop(['index', 'Community Area Number', 'COMMUNITY AREA NAME', 'Date'],axis=1),
        data_sl[['year', 'month', 'day', 'hour', 'HARDSHIP INDEX']],
        data_sl['index'],
        test_size = 0.20,
        random_state = 1551
)

In [16]:
def find_best_params(classifier, parameters, data_x_train, data_y_train):
  pipeline = Pipeline([('sclr', MinMaxScaler()), ('clf', classifier)])
  grid = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=2, scoring='f1_micro')
  grid.fit(data_x_train, data_y_train)
  return grid

In [23]:
parameters = {'clf__criterion': ['entropy'],
 'clf__max_depth': [10],
 'clf__min_samples_leaf': [2],
 'clf__min_samples_split': [25],
 'clf__min_weight_fraction_leaf': [0.0],
 'clf__splitter': ['best']}

grid = find_best_params(DecisionTreeClassifier(), parameters, data_x_train, data_y_train)
grid.best_params_


Fitting 5 folds for each of 1 candidates, totalling 5 fits


{'clf__criterion': 'entropy',
 'clf__max_depth': 10,
 'clf__min_samples_leaf': 2,
 'clf__min_samples_split': 25,
 'clf__min_weight_fraction_leaf': 0.0,
 'clf__splitter': 'best'}

In [17]:
def launch(classifier, parameters, data_x_train, data_y_train, data_x_test, data_y_test, name):
  grid = find_best_params(classifier, parameters, data_x_train, data_y_train)
  estimator = grid.best_estimator_
  print("LAUNCH", name)
  print("TEST")
  prediction = estimator.predict(data_x_test)
  print(classification_report(data_y_test, prediction))
  print("TRAIN")
  prediction = estimator.predict(data_x_train)
  print(classification_report(data_y_train, prediction))
  print("-----")

In [18]:
parameters = {'clf__criterion': ['entropy'],
 'clf__max_depth': [10],
 'clf__min_samples_leaf': [2],
 'clf__min_samples_split': [25],
 'clf__min_weight_fraction_leaf': [0.0],
 'clf__splitter': ['best']}
launch(DecisionTreeClassifier(), parameters, data_x_train, data_y_train, data_x_test, data_y_test, "DecisionTreeClassifier")

parameters = {'clf__criterion': ['gini'],
 'clf__max_depth': [30],
 'clf__min_samples_leaf': [2],
 'clf__min_samples_split': [30],
 'clf__min_weight_fraction_leaf': [0.0],
 'clf__n_estimators': [100]}
launch(RandomForestClassifier(), parameters, data_x_train, data_y_train, data_x_test, data_y_test, "RandomForestClassifier")

parameters = {'clf__criterion': ['friedman_mse'],
 'clf__learning_rate': [0.2],
 'clf__loss': ['deviance'],
 'clf__max_depth': [3],
 'clf__min_samples_leaf': [3],
 'clf__min_samples_split': [10],
 'clf__n_estimators': [125]}
launch(GradientBoostingClassifier(), parameters, data_x_train, data_y_train, data_x_test, data_y_test, "GradientBoostingClassifier")

parameters = {'clf__learning_rate': [0.1],
 'clf__loss': ['auto'],
 'clf__max_iter': [75],
 'clf__max_leaf_nodes': [40],
 'clf__min_samples_leaf': [20]}
launch(HistGradientBoostingClassifier(), parameters, data_x_train, data_y_train, data_x_test, data_y_test, "HistGradientBoostingClassifier")

parameters = {'clf__activation': ['relu'],
 'clf__alpha': [0.0002],
 'clf__learning_rate': ['constant'],
 'clf__learning_rate_init':[0.002],
 'clf__max_iter': [250],
 'clf__solver': ['lbfgs']}
launch(MLPClassifier(), parameters, data_x_train, data_y_train, data_x_test, data_y_test, "MLPClassifier")

parameters = {'clf__C': [1.5],
 'clf__dual': [False],
 'clf__loss': ['squared_hinge'],
 'clf__max_iter': [750],
 'clf__penalty': ['l1']}
launch(LinearSVC(), parameters, data_x_train, data_y_train, data_x_test, data_y_test, "LinearSVC")

parameters = {'clf__algorithm': ['SAMME.R'],
 'clf__base_estimator': [None],
 'clf__learning_rate': [1.5],
 'clf__n_estimators': [200]}
launch(AdaBoostClassifier(), parameters, data_x_train, data_y_train, data_x_test, data_y_test, "AdaBoostClassifier")

Fitting 5 folds for each of 1 candidates, totalling 5 fits
LAUNCH DecisionTreeClassifier
TEST
              precision    recall  f1-score   support

           0       0.32      0.26      0.28     10218
           1       0.36      0.26      0.31     10151
           2       0.40      0.59      0.48     10128
           3       0.43      0.44      0.44     10040

    accuracy                           0.39     40537
   macro avg       0.38      0.39      0.38     40537
weighted avg       0.38      0.39      0.38     40537

TRAIN
              precision    recall  f1-score   support

           0       0.34      0.27      0.30     40453
           1       0.38      0.27      0.32     40520
           2       0.41      0.60      0.49     40543
           3       0.45      0.45      0.45     40631

    accuracy                           0.40    162147
   macro avg       0.39      0.40      0.39    162147
weighted avg       0.39      0.40      0.39    162147

-----
Fitting 5 folds for each

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html



LAUNCH MLPClassifier
TEST
              precision    recall  f1-score   support

           0       0.34      0.18      0.23     10218
           1       0.33      0.25      0.28     10151
           2       0.34      0.53      0.42     10128
           3       0.33      0.40      0.36     10040

    accuracy                           0.34     40537
   macro avg       0.34      0.34      0.32     40537
weighted avg       0.34      0.34      0.32     40537

TRAIN
              precision    recall  f1-score   support

           0       0.34      0.18      0.24     40453
           1       0.33      0.25      0.28     40520
           2       0.35      0.53      0.42     40543
           3       0.34      0.41      0.37     40631

    accuracy                           0.34    162147
   macro avg       0.34      0.34      0.33    162147
weighted avg       0.34      0.34      0.33    162147

-----
Fitting 5 folds for each of 1 candidates, totalling 5 fits
LAUNCH LinearSVC
TEST
           

In [19]:
kmeans = KMeans(n_clusters=4).fit(data_sl[['year', 'month', 'day', 'hour', 'HARDSHIP INDEX', 'Community Area Number']])
kmeans.labels_

array([3, 0, 3, ..., 3, 3, 1], dtype=int32)

In [20]:
to_draw = data_sl[(data_sl['year'] == 2011) & (data_sl['month'] == 2)]

px.scatter_3d(pd.concat(
     [
     to_draw.reset_index(drop=True),
     pd.DataFrame(kmeans.predict(to_draw[['year', 'month', 'day', 'hour', 'HARDSHIP INDEX', 'Community Area Number']]))
     ],
     axis = 1)
, x='day', y='hour', z='HARDSHIP INDEX', color=0)

In [21]:
pdc = pd.concat(
      [
      to_draw.reset_index(drop=True),
      pd.DataFrame(kmeans.predict(to_draw[['year', 'month', 'day', 'hour', 'HARDSHIP INDEX', 'Community Area Number']]))
      ], 
      axis = 1)

px.scatter_3d(pdc, x='day', y='hour', z='HARDSHIP INDEX', color='index')

In [28]:
estimator = grid.best_estimator_
pdc = pd.concat([to_draw.reset_index(drop=True),pd.DataFrame(estimator.predict(to_draw[['year', 'month', 'day', 'hour', 'HARDSHIP INDEX']]))], axis = 1)
px.scatter_3d(pdc, x='day', y='hour', z='HARDSHIP INDEX', color=0)

In [25]:
px.scatter_3d(pd.concat([to_draw.reset_index(drop=True),pd.DataFrame(estimator.predict(to_draw[['year', 'month', 'day', 'hour', 'HARDSHIP INDEX']]))], axis = 1)
, x='day', y='hour', z='Community Area Number', color=0)

In [26]:
px.scatter_3d(to_draw
, x='day', y='hour', z='Community Area Number', color='index')

In [27]:
px.scatter_3d(pd.concat([to_draw.reset_index(drop=True),pd.DataFrame(kmeans.predict(to_draw[['year', 'month', 'day', 'hour', 'HARDSHIP INDEX', 'Community Area Number']]))], axis = 1)
, x='day', y='hour', z='Community Area Number', color=0)